# task11

## 任务四：显卡日志

In [113]:
import pandas as pd
import numpy as np
df_txt = pd.read_table('C:\\datawhale\\joyful-pandas-master\\data\\exercise\\benchmark.txt')[9:-2]
df_txt.index=np.arange(0,df_txt.shape[0],1)
df_txt1=df_txt[::2]#df_txt.index%2==0
df_txt2=df_txt[1::2]
s1=df_txt1.start.str.split('\s',expand=True)[[1,2,5]]
s2=df_txt2.start.str.split('\s',expand=True)[8]
s2.index=np.arange(0,s2.shape[0],1)
s1.index=np.arange(0,s1.shape[0],1)
data=pd.concat([s1,s2],1)
data[3]=s1[1].str.cat(s1[2],sep='-')
data[5]='model_'+data[5]
data.columns=[1,2,'model','time','status']
data['time']=data['time'].astype(float).apply(lambda x:round(x,3))
res=data.pivot(index='model', columns='status', values='time')
res.columns.name=''
res.index.name=''
res.sort_index().head()

,Inference-double,Inference-float,Inference-half,Training-double,Training-float,Training-half
,,,,,,
model_densenet121,144.111,15.637,19.772,417.207,93.357,88.976
model_densenet161,511.177,31.750,27.555,1290.287,136.624,144.319
model_densenet169,175.808,21.598,26.371,511.404,104.840,121.556
model_densenet201,223.960,26.169,33.394,654.365,129.334,118.940
model_mnasnet0_5,11.870,8.039,6.929,48.232,28.528,27.198


## 任务五：水压站点的特征工程

In [114]:
import pandas as pd
import numpy as np
df1 = pd.read_csv('C:\\datawhale\\joyful-pandas-master\\data\\exercise\\yali18.csv')#18年管道压力
df2 = pd.read_csv('C:\\datawhale\\joyful-pandas-master\\data\\exercise\\yali19.csv')#19年管道压力
df3 = pd.read_csv('C:\\datawhale\\joyful-pandas-master\\data\\exercise\\qx1819.csv')#1819年天气

- 通过df1和df2构造df，把时间设为索引，第一列为站点编号，第二列为对应时刻的压力大小

In [115]:
df1=df1.rename(columns={'H%d'%i:'{:0>2d}:00:00'.format(i) for i in range(0,24)})#将列名H0-H23替换为00：00：00这种形式
df2=df2.rename(columns={'H%d'%i:'{:0>2d}:00:00'.format(i) for i in range(0,24)})

In [116]:
df1_melted = df1.melt(id_vars = ['Time', 'MeasName'],value_vars = ['{:0>2d}:00:00'.format(i) for i in range(0,24)],var_name = 'hours',value_name = '压力')
#宽表变长表
df1_melted['datetime']=df1_melted['Time']+' '+df1_melted['hours']
#日期与时刻连接起来
df1_melted['MeasName']=df1_melted['MeasName'].apply(lambda x:x.strip('站点')).astype(int)
#站点的去掉'站点'两个字并将格式转化成int
df1_melted2=df1_melted.drop(columns=['Time','hours'])
#删除日期与时刻两列，它们的信息都在datetime里
df1_temp=df1_melted2.sort_values(by=['datetime','MeasName'],ascending=True).rename(columns={'MeasName':'站点'})
#根据时间和站点排序

In [117]:
#对df2做类似与上面df1的同样处理
df2_melted = df2.melt(id_vars = ['Time', 'MeasName'],value_vars = ['{:0>2d}:00:00'.format(i) for i in range(0,24)],var_name = 'hours',value_name = '压力')
df2_melted['datetime']=df2_melted['Time']+' '+df2_melted['hours']
df2_melted['MeasName']=df2_melted['MeasName'].apply(lambda x:x.strip('站点')).astype(int)
df2_melted2=df2_melted.drop(columns=['Time','hours'])
df2_temp=df2_melted2.sort_values(by=['datetime','MeasName'],ascending=True).rename(columns={'MeasName':'站点'})

In [118]:
df=pd.concat([df1_temp, df2_temp],axis=0)#两个表纵向拼接
df.index=np.arange(0,df.shape[0],1)#将数字索引重新整理成顺序的12345...
index_value = list(df[df['压力'] == -99999].index)#压力有缺测-99999,用前一个顶替
for i in index_value:
    value = df.loc[(i - 1),'压力']
    df.loc[i, '压力'] = value
    
df=df.set_index('datetime')
df.index=pd.to_datetime(df.index)#将datetime列设为索引并转化成datetime对象
df.index.name=''#删除索引名字
df.head(3)

,站点,压力
,,
2018-01-01,1,0.288625
2018-01-01,2,0.317750
2018-01-01,3,0.301375


- 在上一问构造的df基础上，构造下面的特征序列或DataFrame，并把它们逐个拼接到df的右侧

1.当天最高温、最低温和它们的温差

2.当天是否有沙暴、是否有雾、是否有雨、是否有雪、是否为晴天

3.选择一种合适的方法度量雨量/下雪量的大小（构造两个序列分别表示二者大小）

4.限制只用4列，对风向进行0-1编码

In [119]:
df_melted=pd.concat([df1_melted, df2_melted],axis=0)
df_melted.index=np.arange(0,df_melted.shape[0],1)
index_value = list(df_melted[df_melted['压力'] == -99999].index)#压力有缺测-99999,用前一个顶替
for i in index_value:
    value = df_melted.loc[(i - 1),'压力']
    df_melted.loc[i, '压力'] = value
df_melted=df_melted.set_index('Time')
df_melted.head(3)#准备好与新列拼接的数组

,MeasName,hours,压力,datetime
Time,,,,
2018-01-01,4,00:00:00,0.402750,2018-01-01 00:00:00
2018-01-01,7,00:00:00,0.214375,2018-01-01 00:00:00
2018-01-01,22,00:00:00,0.247000,2018-01-01 00:00:00


In [120]:
#准备好最高温度最低温度和温差
df3['最高温']=df3['气温'].str.split('～',expand=True)[0].str.strip().str.strip('C').str.strip('℃').replace('',np.nan)
df3['最低温']=df3['气温'].str.split('～',expand=True)[1].str.strip().str.strip('C').str.strip('℃').replace('',np.nan)
df3=df3.dropna()#最高温最低温有缺测值
df3['最高温']=df3['最高温'].astype('int')
df3['最低温']=df3['最低温'].astype('int')
df3['温差']=df3['最高温']-df3['最低温']
df3=df3.set_index('日期')
#df3.index=pd.to_datetime(df3.index)
df3.head(3)

,天气,气温,风向,最高温,最低温,温差
日期,,,,,,
2018-01-01,多云,1C～-4C,东南风 微风,1,-4,5
2018-01-02,阴转多云,8C～0C,东北风 3-4级,8,0,8
2018-01-03,阴转小雪,1C～-1C,东北风 4-5级转4-5级,1,-1,2


In [121]:
#准备这些量
df3[['沙暴','雾','雨','雪','晴天','雨量','雪量','东风','南风','西风','北风']]=0

In [122]:
df3['沙暴']=df3['沙暴'].mask(df3['天气'].str.contains('沙'),1)
df3['雾']=df3['雾'].mask(df3['天气'].str.contains('雾'),1)
df3['雨']=df3['雨'].mask(df3['天气'].str.contains('雨'),1)
df3['雪']=df3['雪'].mask(df3['天气'].str.contains('雪'),1)
df3['晴天']=df3['晴天'].mask(df3['天气'].str.contains('雪'),1)

In [123]:
df3.head(3)

,天气,气温,风向,最高温,最低温,温差,沙暴,雾,雨,雪,晴天,雨量,雪量,东风,南风,西风,北风
日期,,,,,,,,,,,,,,,,,
2018-01-01,多云,1C～-4C,东南风 微风,1,-4,5,0,0,0,0,0,0,0,0,0,0,0
2018-01-02,阴转多云,8C～0C,东北风 3-4级,8,0,8,0,0,0,0,0,0,0,0,0,0,0
2018-01-03,阴转小雪,1C～-1C,东北风 4-5级转4-5级,1,-1,2,0,0,0,1,1,0,0,0,0,0,0


In [124]:
df_snow=df3['天气'][df3['天气'].str.contains('雪')].str.split('转',expand=True)
weather_snow={'阴':0,'多云':0,'扬沙':0,'雾':0,'晴':0,'小雪':1,'小到中雪':1.5,'雨夹雪':0.5,'大雪':3,'中到大雪':2.5}
df3['雪量']=df3['雪量'].mask(df3['天气'].str.contains('雪'),(df_snow[0].replace(weather_snow)+df_snow[1].replace(weather_snow))/2)
df_rain=df3['天气'][df3['天气'].str.contains('雨')].str.split('转',expand=True)
weather_rain={'阴':0,'多云':0,'扬沙':0,'雾':0,'晴':0,'大雪':0,None:0,'小雨':1,'小到中雨':1.5,'雨':2,'中雨':2,'雨夹雪':0.5,'大雨':3,'中到大雨':2.5,'阵雨':1,'雷阵雨':2,'暴雨':4}
df3['雨量']=df3['雨量'].mask(df3['天气'].str.contains('雨'),(df_rain[0].replace(weather_rain)+df_rain[1].replace(weather_rain))/2)

In [125]:
df3.head(3)

,天气,气温,风向,最高温,最低温,温差,沙暴,雾,雨,雪,晴天,雨量,雪量,东风,南风,西风,北风
日期,,,,,,,,,,,,,,,,,
2018-01-01,多云,1C～-4C,东南风 微风,1,-4,5,0,0,0,0,0,0.0,0.0,0,0,0,0
2018-01-02,阴转多云,8C～0C,东北风 3-4级,8,0,8,0,0,0,0,0,0.0,0.0,0,0,0,0
2018-01-03,阴转小雪,1C～-1C,东北风 4-5级转4-5级,1,-1,2,0,0,0,1,1,0.0,0.5,0,0,0,0


In [126]:
s=df3['风向'].str.split(' ',expand=True)[0]
df3['东风']=df3['东风'].mask(s.str.contains('东'),1)
df3['南风']=df3['南风'].mask(s.str.contains('南'),1)
df3['西风']=df3['西风'].mask(s.str.contains('西'),1)
df3['北风']=df3['北风'].mask(s.str.contains('北'),1)

In [127]:
df3.head(3)

,天气,气温,风向,最高温,最低温,温差,沙暴,雾,雨,雪,晴天,雨量,雪量,东风,南风,西风,北风
日期,,,,,,,,,,,,,,,,,
2018-01-01,多云,1C～-4C,东南风 微风,1,-4,5,0,0,0,0,0,0.0,0.0,1,1,0,0
2018-01-02,阴转多云,8C～0C,东北风 3-4级,8,0,8,0,0,0,0,0,0.0,0.0,1,0,0,1
2018-01-03,阴转小雪,1C～-1C,东北风 4-5级转4-5级,1,-1,2,0,0,0,1,1,0.0,0.5,1,0,0,1


In [128]:
#将天气属性与水管属性拼接
temp22=df_melted.join(df3[['沙暴','雾','雨','雪','晴天','雨量','雪量','东风','南风','西风','北风']], how='left')
res2=temp22.reset_index().drop(columns=['index','hours']).sort_values(by=['datetime','MeasName'],ascending=True).rename(columns={'MeasName':'站点'}).set_index('datetime')
res2.head(3)

,站点,压力,沙暴,雾,雨,雪,晴天,雨量,雪量,东风,南风,西风,北风
datetime,,,,,,,,,,,,,
2018-01-01 00:00:00,1,0.288625,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
2018-01-01 00:00:00,2,0.317750,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
2018-01-01 00:00:00,3,0.301375,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0


- 对df的水压一列构造如下时序特征：

当前时刻该站点水压与本月的相同整点时间该站点水压均值的差，例如当前时刻为2018-05-20 17:00:00，那么对应需要减去的值为当前月所有17:00:00时间点水压值的均值

当前时刻所在周的周末该站点水压均值与工作日水压均值之差

当前时刻向前7日内，该站点水压的均值、标准差、0.95分位数、下雨天数与下雪天数的总和

当前时刻向前7日内，该站点同一整点时间水压的均值、标准差、0.95分位数

当前时刻所在日的该站点水压最高值与最低值出现时刻的时间差

- a

In [129]:
res2.index=pd.to_datetime(res2.index)

In [130]:
res2['hours']=res2.index.hour
res2['month']=res2.index.month
res2['year']=res2.index.year
gpres2=res2.groupby(['year','month','hours'])['压力'].mean()
res3=res2.merge(gpres2, on=['year','month','hours'], how='left')
res3.index=res2.index
res3=res3.rename(columns={'压力_y':'ave_压力','压力_x':'压力'})#对列名重命名
res3['abn1_压力']=res3['压力']-res3['ave_压力']#abn1表示第一问要求的差值
res3.head(3)

,站点,压力,沙暴,雾,雨,雪,晴天,雨量,雪量,东风,南风,西风,北风,hours,month,year,ave_压力,abn1_压力
datetime,,,,,,,,,,,,,,,,,,
2018-01-01,1,0.288625,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0,1,2018,0.264717,0.023908
2018-01-01,2,0.317750,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0,1,2018,0.264717,0.053033
2018-01-01,3,0.301375,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0,1,2018,0.264717,0.036658


- b

In [131]:
#添加一个dayofweek指标，即这天是一周的第几天
res3['weekday']=0
res3=res3.reset_index()
res3['weekday']=res3['weekday'].mask(pd.Series(res3.datetime).dt.dayofweek.isin([5,6]),1)
res3.head(3)

,datetime,站点,压力,沙暴,雾,雨,雪,晴天,雨量,雪量,东风,南风,西风,北风,hours,month,year,ave_压力,abn1_压力,weekday
0,2018-01-01,1,0.288625,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0,1,2018,0.264717,0.023908,0
1,2018-01-01,2,0.317750,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0,1,2018,0.264717,0.053033,0
2,2018-01-01,3,0.301375,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0,1,2018,0.264717,0.036658,0


In [132]:
#添加一个week指标，将这两年分成104个周
len=int((res3.shape[0]-2*30*24)/(7*30*24))
res3['week']=np.hstack((np.array([[i]*7*30*24 for i in range(0,len)]).reshape(-1),np.array([len]*2*30*24)))
s0=pd.DataFrame(res3.groupby(['weekday','week','站点'])['压力'].mean()[0])#依据week指标，周末指标和站点分组
s0.columns=['压力0']
s1=pd.DataFrame(res3.groupby(['weekday','week','站点'])['压力'].mean()[1])#周末
s1.columns=['压力1']
s=s0.join(s1,how='left')#根据索引连接
s['abn2_压力']=s['压力1']-s['压力0']#abn2是第二问的差值
s=s.reset_index().drop(columns=['压力1','压力0'])
res4=res3.merge(s, on=['week','站点'], how='left').set_index('datetime')#依据周和站点将较少的abn2插入到原表中
res4.head(3)

,站点,压力,沙暴,雾,雨,雪,晴天,雨量,雪量,东风,...,西风,北风,hours,month,year,ave_压力,abn1_压力,weekday,week,abn2_压力
datetime,,,,,,,,,,,,,,,,,,,,,
2018-01-01,1,0.288625,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0,1,2018,0.264717,0.023908,0,0,-0.001511
2018-01-01,2,0.317750,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0,1,2018,0.264717,0.053033,0,0,-0.002209
2018-01-01,3,0.301375,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0,1,2018,0.264717,0.036658,0,0,-0.001914


- c：当前时刻向前7日内，该站点水压的均值、标准差、0.95分位数、下雨天数与下雪天数的总和

In [133]:
import warnings
warnings.filterwarnings("ignore")

smean=res4.groupby(['站点','datetime'])['压力'].mean()#依据站点和日期分组
s_temp=pd.DataFrame(index=smean.index.get_level_values(1),columns=['7_ave_压力','7_std_压力','7_q95_压力','站点'],dtype=float)
s_temp['站点']=smean.index.get_level_values(0)
for i in range(1,31):#对于30个站点
    idx=s_temp['站点']==i
    s_temp['7_ave_压力'][idx]=smean[i].rolling(7*24).mean()
    s_temp['7_std_压力'][idx]=smean[i].rolling(7*24).std()
    s_temp['7_q95_压力'][idx]=smean[i].rolling(7*24).quantile(0.95)
    #s_temp就容纳了所有值
    
s_temp

,7_ave_压力,7_std_压力,7_q95_压力,站点
datetime,,,,
2018-01-01 00:00:00,NaN,NaN,NaN,1
2018-01-01 01:00:00,NaN,NaN,NaN,1
2018-01-01 02:00:00,NaN,NaN,NaN,1
2018-01-01 03:00:00,NaN,NaN,NaN,1
2018-01-01 04:00:00,NaN,NaN,NaN,1
...,...,...,...,...
2019-12-31 19:00:00,0.276701,0.027157,0.319988,30
2019-12-31 20:00:00,0.276730,0.027151,0.319988,30
2019-12-31 21:00:00,0.276777,0.027118,0.319988,30


In [134]:
s_temp=s_temp.reset_index().drop(columns=['站点','datetime'])
res4=pd.concat([res4.reset_index(),s_temp],1,join='outer')

In [135]:
res4=res4.set_index('datetime')

In [136]:
#下雨下雪的日子
df3['雨雪']=0
df3['雨雪']=df3['雨雪'].mask(df3['雨']==1,1).mask(df3['雪']==1,1)

In [137]:
s=df3['雨雪'].rolling(7).sum()
s=s.reset_index()
s['日期']=pd.to_datetime(s['日期'])
s=s.set_index('日期')
res4['日期']=res4.index.date
res4.index=res4.index.date

In [138]:
res4=res4.join(s,how='left')
res4.head(3)

,站点,压力,沙暴,雾,雨,雪,晴天,雨量,雪量,东风,...,ave_压力,abn1_压力,weekday,week,abn2_压力,7_ave_压力,7_std_压力,7_q95_压力,日期,雨雪
2018-01-01,1,0.288625,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.264717,0.023908,0,0,-0.001511,NaN,NaN,NaN,2018-01-01,NaN
2018-01-01,2,0.317750,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.264717,0.053033,0,0,-0.002209,NaN,NaN,NaN,2018-01-01,NaN
2018-01-01,3,0.301375,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.264717,0.036658,0,0,-0.001914,NaN,NaN,NaN,2018-01-01,NaN


- d：当前时刻向前7日内，该站点同一整点时间水压的均值、标准差、0.95分位数

In [139]:
res4['7_ave_压力2']=res4.rolling('7D')['压力'].mean()
res4['7_std_压力2']=res4.rolling('7D')['压力'].std()
res4['7_q95_压力2']=res4.rolling('7D')['压力'].quantile(0.95)
res4.head(3)

,站点,压力,沙暴,雾,雨,雪,晴天,雨量,雪量,东风,...,week,abn2_压力,7_ave_压力,7_std_压力,7_q95_压力,日期,雨雪,7_ave_压力2,7_std_压力2,7_q95_压力2
2018-01-01,1,0.288625,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0,-0.001511,NaN,NaN,NaN,2018-01-01,NaN,0.288625,NaN,0.288625
2018-01-01,2,0.317750,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0,-0.002209,NaN,NaN,NaN,2018-01-01,NaN,0.303188,0.020594,0.316294
2018-01-01,3,0.301375,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0,-0.001914,NaN,NaN,NaN,2018-01-01,NaN,0.302583,0.014600,0.316113


- e:当前时刻所在日的该站点水压最高值与最低值出现时刻的时间差

In [140]:
def my_func(x):
    value=abs(x.argmax()-x.argmin())
    return(pd.Series('0 days %d hours'%value))
se=res4.groupby(['站点','日期'])['压力'].apply(lambda x:my_func(x))
se=se.reset_index().rename(columns={'压力':'时间差'}).drop(columns='level_2')

In [141]:
se

,站点,日期,时间差
0,1,2018-01-01,0 days 6 hours
1,1,2018-01-02,0 days 6 hours
2,1,2018-01-03,0 days 18 hours
3,1,2018-01-04,0 days 7 hours
4,1,2018-01-05,0 days 8 hours
...,...,...,...
21895,30,2019-12-27,0 days 5 hours
21896,30,2019-12-28,0 days 6 hours
21897,30,2019-12-29,0 days 17 hours
21898,30,2019-12-30,0 days 8 hours


In [142]:
res4=res4.merge(se,on=['站点','日期'], how='left')

In [143]:
res4['时间差']=pd.to_timedelta(res4['时间差'].values)

In [144]:
res4.head()

,站点,压力,沙暴,雾,雨,雪,晴天,雨量,雪量,东风,...,abn2_压力,7_ave_压力,7_std_压力,7_q95_压力,日期,雨雪,7_ave_压力2,7_std_压力2,7_q95_压力2,时间差
0,1,0.288625,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,-0.001511,NaN,NaN,NaN,2018-01-01,NaN,0.288625,NaN,0.288625,0 days 06:00:00
1,2,0.317750,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,-0.002209,NaN,NaN,NaN,2018-01-01,NaN,0.303188,0.020594,0.316294,0 days 14:00:00
2,3,0.301375,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,-0.001914,NaN,NaN,NaN,2018-01-01,NaN,0.302583,0.014600,0.316113,0 days 06:00:00
3,4,0.402750,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,-0.004297,NaN,NaN,NaN,2018-01-01,NaN,0.327625,0.051482,0.390000,0 days 03:00:00
4,5,0.314625,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,-0.002105,NaN,NaN,NaN,2018-01-01,NaN,0.325025,0.044963,0.385750,0 days 06:00:00
